In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import pickle

In [2]:
df_location = pd.read_excel('../../python/df_location.xlsx')
df_distance = pd.read_excel('../../python/df_distance_km.xlsx')
df_vehicle = pd.read_excel('../../python/df_vehicle.xlsx')
df_orders = pd.read_excel('../../python/df_orders.xlsx')

In [3]:
clients_names = df_location['cliente'].to_list()
clients_coords = df_location[['Latitud','Longitud']].to_numpy()

clients = {name: coord for name, coord in zip(clients_names, clients_coords)}

In [4]:
distances_matrix = df_distance.to_numpy()

In [5]:
def calculate_route_distance(route, distance_matrix, cost_per_unit):
    distance = 0
    for i in range(len(route) - 1):
        dist = distance_matrix[route[i]][route[i + 1]]
        if dist == 0:
            return float('inf')
        distance += dist * cost_per_unit
    return distance

In [6]:
def is_valid_route(route, demandas, capacity):
    total_demand = sum(demandas[cliente] for cliente in route)
    return total_demand <= capacity

In [7]:
def generate_valid_initial_routes(distance_matrix, num_vehiculos, almacen, demandas, capacidades):
    clientes = [cliente for cliente in range(len(distance_matrix)) if cliente != almacen]
    valid_routes_found = False
    while not valid_routes_found:
        random.shuffle(clientes)
        routes = [[almacen] for _ in range(num_vehiculos)]
        for cliente in clientes:
            for route in routes:
                if sum(demandas[r] for r in route) + demandas[cliente] <= capacidades[routes.index(route)]:
                    route.append(cliente)
                    break
        for route in routes:
            route.append(almacen)
        if all(calculate_route_distance(route, distance_matrix, 1) != float('inf') for route in routes):
            valid_routes_found = True
    return routes


In [8]:
def calculate_total_distance(routes, distance_matrix, costes):
    total_distance = 0
    for route, cost in zip(routes, costes):
        total_distance += calculate_route_distance(route, distance_matrix, cost)
    return total_distance

In [9]:
def generate_vecinos_multiple(routes, almacen, demandas, capacidades):
    neighbors = []
    for i in range(len(routes)):
        for j in range(len(routes)):
            if i != j:
                for k in range(1, len(routes[i]) - 1):
                    for l in range(1, len(routes[j]) - 1):
                        neighbor = [route[:] for route in routes]
                        neighbor[i][k], neighbor[j][l] = neighbor[j][l], neighbor[i][k]
                        if is_valid_route(neighbor[i], demandas, capacidades[i]) and is_valid_route(neighbor[j], demandas, capacidades[j]):
                            neighbors.append(neighbor)
    return neighbors

In [10]:
def tabu_search_multiple_tsp(distance_matrix, initial_routes, max_iterations, tabu_size, almacen, costes, demandas, capacidades, max_no_improve=20):
    current_routes = initial_routes
    best_routes = current_routes
    best_distance = calculate_total_distance(current_routes, distance_matrix, costes)
    tabu_list = []
    historial = []
    no_improve_counter = 0

    for iteration in range(max_iterations):
        neighbors = generate_vecinos_multiple(current_routes, almacen, demandas, capacidades)
        evaluated_neighbors = [(neighbor, calculate_total_distance(neighbor, distance_matrix, costes)) 
                               for neighbor in neighbors if neighbor not in tabu_list]

        if not evaluated_neighbors:
            break  # Si no hay vecinos válidos, detener el algoritmo

        best_neighbor, best_neighbor_distance = min(evaluated_neighbors, key=lambda x: x[1])
        
        current_routes = best_neighbor
        if best_neighbor_distance < best_distance:
            best_routes = best_neighbor
            best_distance = best_neighbor_distance
            no_improve_counter = 0
        else:
            no_improve_counter += 1 

        # Actualizar lista Tabú
        tabu_list.append(best_neighbor)
        if len(tabu_list) > tabu_size:
            tabu_list.pop(0)

        # Guardar historial
        historial.append((best_neighbor, best_neighbor_distance))

        # Verificar criterio de parada adicional
        if no_improve_counter >= max_no_improve:
            print(f"Parada anticipada: {no_improve_counter} iteraciones sin mejora.")
            break

    return best_routes, best_distance, historial



In [11]:
almacen = len(clients_names) - 1
num_vehicle = 5
costes = df_vehicle['costo_km'].to_list()
capacidades = df_vehicle['capacidad_kg'].to_list()
demandas_dict = dict(zip(df_orders["cliente"], df_orders["order_demand"]))
demandas = [demandas_dict.get(client, 0) for client in clients_names]

initial_routes = generate_valid_initial_routes(distances_matrix, num_vehicle, almacen, demandas, capacidades)


In [12]:
# Parámetros iniciales
max_iterations = 100
tabu_size = 10
max_no_improve = 20  # Número máximo de iteraciones sin mejora

# Ejecutar Tabu Search
mejor_solucion, costo_mejor_solucion, historial = tabu_search_multiple_tsp(
    distances_matrix, initial_routes, max_iterations, tabu_size, almacen, costes, demandas, capacidades, max_no_improve=max_no_improve
)


Parada anticipada: 20 iteraciones sin mejora.


In [13]:
def convert_routes_to_coordinates(routes, clients):
    routes_with_coordinates = []
    for route in routes:
        route_coords = [tuple(clients[clients_names[client]]) for client in route]
        routes_with_coordinates.append(route_coords)
    return routes_with_coordinates

mejor_solucion_coords = convert_routes_to_coordinates(mejor_solucion, clients)

In [14]:
# Guardar el modelo
modelo_tabu = {
    "mejor_solucion": mejor_solucion_coords,
    "costo_mejor_solucion": costo_mejor_solucion,
    "parametros": {
        "tamano_tabu": tabu_size,
        "max_iteraciones": max_iterations,
        "criterio_parada": f"Sin mejora en {max_no_improve} iteraciones"
    },
    "historial": historial
}

In [15]:
print(modelo_tabu)

{'mejor_solucion': [[(np.float64(40.3885963), np.float64(-3.7270384)), (np.float64(40.3719903), np.float64(-3.6950556)), (np.float64(40.3340239), np.float64(-3.7118486)), (np.float64(40.3885963), np.float64(-3.7270384))], [(np.float64(40.3885963), np.float64(-3.7270384)), (np.float64(40.5124455), np.float64(-3.6778586)), (np.float64(40.4343313), np.float64(-3.7045345)), (np.float64(40.4718405), np.float64(-3.7080787)), (np.float64(40.4678501), np.float64(-3.8145949)), (np.float64(40.3885963), np.float64(-3.7270384))], [(np.float64(40.3885963), np.float64(-3.7270384)), (np.float64(40.4090831), np.float64(-3.676125)), (np.float64(40.4117919), np.float64(-3.6593671)), (np.float64(40.4468717), np.float64(-3.5865153)), (np.float64(40.4347103), np.float64(-3.6422656)), (np.float64(40.4265429), np.float64(-3.6080509)), (np.float64(40.3885963), np.float64(-3.7270384))], [(np.float64(40.3885963), np.float64(-3.7270384)), (np.float64(40.3893878), np.float64(-3.7004158)), (np.float64(40.3862643),

In [16]:
with open('modelo_tabu.pkl', 'wb') as archivo:
    pickle.dump(modelo_tabu, archivo)